# Sample met data for testing ModVege

Data used: Met Eireann daily data for Valentia Observatory, Co. Kerry
(<https://data.gov.ie/dataset/valentia-observatory-daily-data>)

In [1]:
import os
from datetime import datetime, timezone
import pandas as pd
import pooch

In [2]:
print("Last updated:", datetime.now(tz=timezone.utc))

Last updated: 2023-01-31 18:04:08.992994+00:00


In [3]:
DATA_DRIVE = "data"
URL = "https://cli.fusio.net/cli/climate_data/webdata/dly2275.csv"
SUB_DIR = os.path.join(DATA_DRIVE, "met", "MetEireann")
KNOWN_HASH = None
os.makedirs(SUB_DIR, exist_ok=True)
FILE_NAME = "dly2275.csv"

In [4]:
# download data if it doesn't exist
if not os.path.isfile(os.path.join(SUB_DIR, FILE_NAME)):
    pooch.retrieve(
        url=URL, known_hash=KNOWN_HASH, fname=FILE_NAME, path=SUB_DIR
    )

    with open(
        os.path.join(SUB_DIR, f"{FILE_NAME[:-4]}.txt"), "w", encoding="utf-8"
    ) as outfile:
        outfile.write(
            f"Data downloaded on: {datetime.now(tz=timezone.utc)}\n"
            f"Download URL: {URL}"
        )

In [5]:
val = pd.read_csv(
    os.path.join(SUB_DIR, FILE_NAME), skiprows=24, parse_dates=["date"]
)

In [6]:
val.head()

,date,ind,maxtp,ind.1,mintp,igmin,gmin,ind.2,rain,cbl,...,ind.5,hg,sun,glorad,soil,pe,evap,smd_wd,smd_md,smd_pd
0,1942-01-01,0,11.1,0,9.8,0.0,8.0,0,1.7,1026.2,...,0,32.0,NaN,NaN,NaN,0.9,1.1,NaN,NaN,NaN
1,1942-01-02,0,11.6,0,10.0,0.0,8.6,0,4.6,1023.8,...,0,33.0,NaN,NaN,NaN,0.6,0.8,NaN,NaN,NaN
2,1942-01-03,0,12.1,0,9.8,0.0,9.4,0,16.7,1014.5,...,0,39.0,NaN,NaN,NaN,0.3,0.5,NaN,NaN,NaN
3,1942-01-04,0,9.9,0,4.0,0.0,6.7,0,3.5,1012.0,...,0,37.0,NaN,NaN,NaN,0.5,0.7,NaN,NaN,NaN
4,1942-01-05,0,4.9,0,1.4,0.0,0.4,0,0.0,1022.5,...,0,45.0,NaN,NaN,NaN,0.9,1.1,NaN,NaN,NaN


In [7]:
# filter data for 2019 - 2021
val = val[(val["date"].dt.year >= 2019) & (val["date"].dt.year <= 2021)]

In [8]:
val.sort_values(by=["date"], inplace=True)
val.reset_index(inplace=True)

In [9]:
# keep only min and max air temps, rainfall, global radiation, PET
val = val[["date", "maxtp", "mintp", "rain", "glorad", "pe"]]

In [10]:
val.head()

,date,maxtp,mintp,rain,glorad,pe
0,2019-01-01,11.4,8.3,0.0,272.0,0.3
1,2019-01-02,10.8,7.9,0.1,299.0,0.5
2,2019-01-03,10.0,7.9,0.9,171.0,0.8
3,2019-01-04,9.2,7.5,0.0,207.0,1.0
4,2019-01-05,10.8,8.3,0.0,122.0,0.9


In [11]:
# calculate mean air temperature
val["T"] = val[["maxtp", "mintp"]].mean(axis=1)

In [12]:
# convert global radiation units to MJ m-2 from J cm-2
# see Allen et al. (1998)
val["RG"] = val["glorad"].astype(float) / 100.0

In [13]:
# convert global radiation to PAR
# see Papaioannou et al. (1993)
val["PAR"] = val["RG"] * 0.473

In [14]:
# rainfall
val["PP"] = val["rain"].astype(float)

In [15]:
# potential evapotranspiration
val["PET"] = val["pe"].astype(float)

In [16]:
val["time"] = val["date"]

In [17]:
# keep only relevant columns
val = val[["time", "T", "PAR", "PET", "PP", "RG"]]

In [18]:
val.head()

,time,T,PAR,PET,PP,RG
0,2019-01-01,9.85,1.28656,0.3,0.0,2.72
1,2019-01-02,9.35,1.41427,0.5,0.1,2.99
2,2019-01-03,8.95,0.80883,0.8,0.9,1.71
3,2019-01-04,8.35,0.97911,1.0,0.0,2.07
4,2019-01-05,9.55,0.57706,0.9,0.0,1.22


In [19]:
val.tail()

,time,T,PAR,PET,PP,RG
1091,2021-12-27,9.60,1.71699,0.3,0.9,3.63
1092,2021-12-28,9.65,0.70004,0.6,7.7,1.48
1093,2021-12-29,12.60,1.19669,0.5,9.9,2.53
1094,2021-12-30,12.70,0.30272,0.3,22.2,0.64
1095,2021-12-31,13.20,0.44462,0.7,2.0,0.94


In [20]:
# save as a CSV file
val.to_csv(os.path.join(SUB_DIR, "valentia.csv"), index=False)